In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# --------------step1: 定义超参数-------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 是否用GPU
EPOCHS = 10  # 数据集的训练次数
BATCH_SIZE = 128  # 每批处理的数据

In [3]:
# -------------step2: 下载并加载数据集------------------
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('mnist-data/', train=True, download=True,transform=transforms.Compose([transforms.ToTensor(),])),
        batch_size=BATCH_SIZE, shuffle=True)# 加载数据集

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('mnist-data/', train=False, transform=transforms.Compose([transforms.ToTensor(),])),
        batch_size=BATCH_SIZE, shuffle=True)# 加载测试集

In [4]:
# -------------step3: 构建网络模型--------------------
class LeNet(nn.Module):
    """
    A neural network with:
      2 Conbolutions
      3 Full connnection
    """
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, (5, 5), (1, ), 2)  # 1:输入通道, 6:输出通道, (5,5):kernel_size, 1:stride, 2:padding
        self.conv2 = nn.Conv2d(6, 16, (5, 5))  # 6:输入通道, 16:输出通道, (5,5):kernel_size
        self.fc1 = nn.Linear(16*5*5, 120)   # 16*5*5:输入通道, 120:输出通道
        self.fc2 = nn.Linear(120, 84)  # 输入通道:120, 输出通道:84
        self.fc3 = nn.Linear(84, 10)  # 输入通道:84, 输出通道:10

    def forward(self, x):
        x = self.conv1(x)  # 输入：batch*1*28*28 输出：batch*6*28*28 （28+2*2-5+1=28）
        x = F.relu(x)      # 激活层 输出：batch**6*28*28
        x = F.max_pool2d(x, 2, 2)  # 池化层/下采样 输入：batch*6*28*28 输出：batch*6*14*14

        x = self.conv2(x)  # 输入：batch*6*14*14 输出：batch*16*10*10  （14+2*0-5+1=10）
        x = F.relu(x)  # 激活层 输出：batch*16*10*10
        x = F.max_pool2d(x, 2, 2)  # 池化层/下采样 输入：batch*16*10*10 输出：16*5*5

        x = x.view(x.size(0), -1)  # 拉平 （-1指自动计算维度） 16*5*5

        x = self.fc1(x)  # 输入：batch*16*5*5  输出： batch*120
        x = F.relu(x)    # 激活层 输出：batch*120
        x = self.fc2(x)  # 输入：batch*120  输出：batch*84
        x = F.relu(x)    # 激活层  输出：batch*84
        x = self.fc3(x)  # 输入：batch*84  输出：batch*10
        output = F.softmax(x, dim=1)  # 计算分类后每个数字的概率值

        return output

In [5]:
# ----------------step:4 定义优化器--------------------------
model = LeNet().to(DEVICE)
optimizer = optim.Adam(model.parameters())

In [6]:
# ----------------step5: 定义训练方法-----------------------
def train_model(my_model, device, trains_loader, optimizers, epoches):
    # 模型训练
    my_model.train()
    for batch_idx, (data, target) in enumerate(trains_loader):
        # 将data和target部署到DEVICE上去
        data, target = data.to(device), target.to(device)
        # 将梯度初始化为0
        optimizers.zero_grad()
        # 训练所得的结果
        output = my_model(data)
        # 计算交叉熵损失
        loss = F.cross_entropy(output, target)
        # 反向传播
        loss.backward()
        # 更新参数
        optimizers.step()
        # 每100batch_size打印一次log
        if batch_idx % 1000 == 0:
            print("Training Epoch:{} \t Loss:{:.5f}".format(epoches, loss.item()))


In [8]:
# ----------------step6: 定义测试方法------------------------
def test_model(my_model, device, test_loder):
    my_model.eval()  # 模型验证
    correct = 0.0    # 正确率
    test_loss = 0.0   # 测试损失
    with torch.no_grad():  # 测试时不计算梯度，也不进行反向传播
        for data, target in test_loder:
            # 将data和target部署到device上
            data, target = data.to(device), target.to(device)
            # 测试所得的结果
            output = my_model(data)
            # 计算交叉熵损失
            test_loss += F.cross_entropy(output, target).item()
            # 找到概率最大的下标
            predict = output.argmax(dim=1)
            # predict = torch.max(output, dim=1)
            correct += predict.eq(target.view_as(predict)).sum().item()  # 累计正确的值
        # 计算平均损失
        avg_loss = test_loss / len(test_loder.dataset)
        # 计算准确率
        correct_ratio = 100 * correct / len(test_loder.dataset)
        print("Average_loss in test:{:.5f}\t Accuracy:{:.5f}\n".format(
            avg_loss, correct_ratio
        ))

In [9]:
%%time
# --------------step7: 训练模型----------------------------
for epoch in range(1, EPOCHS+1):
    train_model(model, DEVICE, train_loader, optimizer, epoch)
    test_model(model, DEVICE, test_loader)

Training Epoch:1 	 Loss:1.61583
Average_loss in test:0.01181	 Accuracy:96.83000

Training Epoch:2 	 Loss:1.50726
Average_loss in test:0.01177	 Accuracy:97.18000

Training Epoch:3 	 Loss:1.50278
Average_loss in test:0.01172	 Accuracy:97.84000

Training Epoch:4 	 Loss:1.49188
Average_loss in test:0.01174	 Accuracy:97.67000

Training Epoch:5 	 Loss:1.46716
Average_loss in test:0.01168	 Accuracy:98.27000

Training Epoch:6 	 Loss:1.47595
Average_loss in test:0.01168	 Accuracy:98.36000

Training Epoch:7 	 Loss:1.47685
Average_loss in test:0.01165	 Accuracy:98.65000

Training Epoch:8 	 Loss:1.48452
Average_loss in test:0.01166	 Accuracy:98.49000

Training Epoch:9 	 Loss:1.46307
Average_loss in test:0.01167	 Accuracy:98.50000

Training Epoch:10 	 Loss:1.46448
Average_loss in test:0.01165	 Accuracy:98.73000

Wall time: 1min 22s
